In [5]:
import src.acquire
#import src.prepare
#import src.explore
#import src.model

In [6]:
# path='zillow_df.csv'

# df, X_train_explore, \
#     X_train_scaled, y_train, \
#     X_validate_scaled, y_validate, \
#     X_test_scaled, y_test = src.prepare.wrangle_zillow(path)

# X_train_explore.shape, X_validate_scaled.shape, X_test_scaled.shape

In [19]:
import pandas as pd
import numpy as np
import scipy as sp 
import os
from env import host, user, password
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, QuantileTransformer, PowerTransformer, RobustScaler, MinMaxScaler

def wrangle_zillow(path):
    df = pd.read_csv(path)
    
    #Columns to drop
    df.drop(columns= ['parcelid','id','airconditioningtypeid','architecturalstyletypeid','basementsqft','buildingclasstypeid','buildingqualitytypeid'], inplace = True)
    df.drop(columns= ['calculatedbathnbr','decktypeid','finishedfloor1squarefeet','finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15'], inplace = True)
    df.drop(columns= ['finishedsquarefeet50','finishedsquarefeet6','fips','fullbathcnt','heatingorsystemtypeid','poolsizesum','pooltypeid10','pooltypeid2'], inplace = True)
    df.drop(columns= ['pooltypeid7','propertycountylandusecode','propertyzoningdesc','rawcensustractandblock','regionidcity','regionidcounty','regionidneighborhood'], inplace = True)
    df.drop(columns= ['storytypeid','threequarterbathnbr','typeconstructiontypeid','unitcnt','yardbuildingsqft17','yardbuildingsqft26','numberofstories'], inplace = True)
    df.drop(columns= ['fireplaceflag','structuretaxvaluedollarcnt','assessmentyear','landtaxvaluedollarcnt','taxamount','taxdelinquencyflag','taxdelinquencyyear'], inplace = True)
    df.drop(columns= ['censustractandblock','logerror','transactiondate','garagetotalsqft','latitude','longitude'], inplace = True)
    
    # Rename columns for clarity
    df.rename(columns={"hashottuborspa":"hottub_spa","fireplacecnt":"fireplace","garagecarcnt":"garage"}, inplace = True)

    # Replaces NaN values with 0
    df['garage'] = df['garage'].replace(np.nan, 0)
    df['hottub_spa'] = df['hottub_spa'].replace(np.nan, 0)
    df['lotsizesquarefeet'] = df['lotsizesquarefeet'].replace(np.nan, 0)
    df['poolcnt'] = df['poolcnt'].replace(np.nan, 0)
    df['fireplace'] = df['fireplace'].replace(np.nan, 0)
        
    # Add dummy variables as new columns in dataframe and rename them, delete origional
    df["regionidzip"] = df["regionidzip"].astype('category')
    
    df["propertylandusetypeid"]= df["propertylandusetypeid"].astype('category')
    
    df["yearbuilt"]= df["yearbuilt"].astype('category')
    

    # Problem 'bedbathratio' - New Feature (Ratio of bedroomcnt and bathroomcnt)
    #df['bedbathratio'] = df.bedroomcnt.div(df.bathroomcnt, axis=0)

#     # split dataset
#     train_validate, test = train_test_split(df, test_size = .2, random_state = 123)
#     train, validate = train_test_split(train_validate, test_size = .3, random_state = 123)
#     train.shape, validate.shape, test.shape

#     # Assign variables
#     # x df's are all numeric cols 
#     X_train_explore = train.drop(columns=['taxvaluedollarcnt'])
#     X_train = train.drop(columns=['taxvaluedollarcnt',"regionidzip","propertylandusetypeid","yearbuilt"])
#     X_validate = validate.drop(columns=['taxvaluedollarcnt',"regionidzip","propertylandusetypeid","yearbuilt"])
#     X_test = test.drop(columns=['taxvaluedollarcnt',"regionidzip","propertylandusetypeid","yearbuilt"])

#     # y df's are just fertility
#     y_train = train[['taxvaluedollarcnt']]
#     y_validate = validate[['taxvaluedollarcnt']]
#     y_test = test[['taxvaluedollarcnt']]

#     scaler = MinMaxScaler(copy=True).fit(X_train)

#     X_train_scaled = scaler.transform(X_train)
#     X_validate_scaled = scaler.transform(X_validate)
#     X_test_scaled = scaler.transform(X_test)

#     X_train_scaled = pd.DataFrame(X_train_scaled, 
#                                 columns=X_train.columns.values).\
#                                 set_index([X_train.index.values])

#     X_validate_scaled = pd.DataFrame(X_validate_scaled, 
#                                     columns=X_validate.columns.values).\
#                                 set_index([X_validate.index.values])

#     X_test_scaled = pd.DataFrame(X_test_scaled, 
#                                     columns=X_test.columns.values).\
#                                 set_index([X_test.index.values])
#     return df, X_train_explore, X_train_scaled, y_train, X_validate_scaled, y_validate, X_test_scaled, y_test
    return df

In [20]:

df = wrangle_zillow(path)
# df, X_train_explore, \
#     X_train_scaled, y_train, \
#     X_validate_scaled, y_validate, \
#     X_test_scaled, y_test = wrangle_zillow(path)

# X_train_explore.shape, X_validate_scaled.shape, X_test_scaled.shape

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20931 entries, 0 to 20930
Data columns (total 15 columns):
 #   Column                        Non-Null Count  Dtype   
---  ------                        --------------  -----   
 0   Unnamed: 0                    20931 non-null  int64   
 1   bathroomcnt                   20931 non-null  float64 
 2   bedroomcnt                    20931 non-null  float64 
 3   calculatedfinishedsquarefeet  20868 non-null  float64 
 4   fireplace                     20931 non-null  float64 
 5   garage                        20931 non-null  float64 
 6   hottub_spa                    20931 non-null  float64 
 7   lotsizesquarefeet             20931 non-null  float64 
 8   poolcnt                       20931 non-null  float64 
 9   propertylandusetypeid         20931 non-null  category
 10  regionidzip                   20916 non-null  category
 11  roomcnt                       20931 non-null  float64 
 12  yearbuilt                     20850 non-null  

In [ ]:


def edit_df(df):
    # Drop Columns
    df.drop(columns= ['parcelid','id','airconditioningtypeid','architecturalstyletypeid','basementsqft','buildingclasstypeid','buildingqualitytypeid'], inplace = True)
    df.drop(columns= ['calculatedbathnbr','decktypeid','finishedfloor1squarefeet','finishedsquarefeet12','finishedsquarefeet13','finishedsquarefeet15'], inplace = True)
    df.drop(columns= ['finishedsquarefeet50','finishedsquarefeet6','fips','fullbathcnt','heatingorsystemtypeid','poolsizesum','pooltypeid10','pooltypeid2'], inplace = True)
    df.drop(columns= ['pooltypeid7','propertycountylandusecode','propertyzoningdesc','rawcensustractandblock','regionidcity','regionidcounty','regionidneighborhood'], inplace = True)
    df.drop(columns= ['storytypeid','threequarterbathnbr','typeconstructiontypeid','unitcnt','yardbuildingsqft17','yardbuildingsqft26','numberofstories'], inplace = True)
    df.drop(columns= ['fireplaceflag','structuretaxvaluedollarcnt','assessmentyear','landtaxvaluedollarcnt','taxamount','taxdelinquencyflag','taxdelinquencyyear'], inplace = True)
    df.drop(columns= ['censustractandblock','logerror','transactiondate','garagetotalsqft'], inplace = True)
    
    # Rename columns for clarity
    df.rename(columns={"hashottuborspa":"hottub_spa","fireplacecnt":"fireplace","garagecarcnt":"garage"}, inplace = True)
    
    # Replaces NaN values with 0
    df['garage'] = df['garage'].replace(np.nan, 0)
    df['hottub_spa'] = df['hottub_spa'].replace(np.nan, 0)
    df['lotsizesquarefeet'] = df['lotsizesquarefeet'].replace(np.nan, 0)
    df['poolcnt'] = df['poolcnt'].replace(np.nan, 0)
    df['fireplace'] = df['fireplace'].replace(np.nan, 0)
    
    # Add dummy variables as new columns in dataframe and rename them, delete origional
    df["regionidzip"] = df["regionidzip"].astype('category')
    dummy_df = df.regionidzip.str.get_dummies()
    df = pd.concat([df, dummy_df], axis=1)
    
    df["propertylandusetypeid"]= df["propertylandusetypeid"].astype('category')
    dummy_df = df.propertylandusetypeid.str.get_dummies()
    df = pd.concat([df, dummy_df], axis=1)
    
    df["yearbuilt"]= df["yearbuilt"].astype('category')
    dummy_df = df.yearbuilt.str.get_dummies()
    df = pd.concat([df, dummy_df], axis=1)
    return df
    

## Notes

1. I already uploaded my env.py file so need need to upload yours.  We only need one to access the data.


2. I already uploaded a .gitignore so also no worries there


3. Data is ready to go, i want to engeneer a feature that combines bed and bath, but get a weird error when I try.    we can trouble shoot after mvp, but code can be found in .prepare file


4. I want to leave 2 categorical variables after transforming them to numeric but I also need help with that
   - "regionidzip"
   - "propertylandusetypeid"
   - They need to be made in to categories before we can do any testing on them (Cat to Numeric)


5. id.1 and Unmaed:0 showd up we also need to delete them